In [1]:
using DataFrames, DataFramesMeta, Optim, Revise, Serialization, DebuggingUtilities, LinearAlgebra, Statistics
using JuMP, Ipopt

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [7]:
test1state = false
if test1state 
    dirpath = "/export/storage_adgandhi/MiscLi/factract/analysis"
else
    dirpath = "jls"
end;
ec = deserialize("$dirpath/ec.jls");

---
### jump

In [8]:
q_obs = ec.q_obs;
tracts = ec.tracts;
n_firms = length(ec.firms);
X = [t.X[:,1:2] for t in tracts];
D = [t.D[:,1:2] for t in tracts];

In [9]:
j_indexer, t_indexer, positionin = m.create_indices(tracts, n_firms);

In [10]:
θx, θd, share, mktq, firmq = m.run_nlls(X, D, tracts, n_firms, q_obs);

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        4
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.2145106e+09 0.00e+00 1.00e+02  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

In [15]:
ηDj = m.compute_elasticity(D, θd, share, mktq, t_indexer, positionin, n_firms);
mean(ηDj)

-3.1206247312851056

In [12]:
X_dc = [t.X[:,1] for t in tracts];
X_rn = [t.X[:,2] for t in tracts];

In [13]:
η_dc = m.compute_elasticity(X_dc, θx[1], share, mktq, t_indexer, positionin, n_firms);
mean(η_dc)

-0.452947792814823

In [14]:
η_rn = m.compute_elasticity(X_rn, θx[2], share, mktq, t_indexer, positionin, n_firms);
mean(η_rn)

-0.13938613658809862